# Qualitative Result

# CONTENTS

1. [SNLI Baseline Analysis](#SNLI-Baseline-Analysis)
2. [SNLI BERT Analysis](#SNLI-BERT-Analysis)
3. [SNLI RoBERTa Analysis](#SNLI-RoBERTa-Analysis)
4. [SNLI ALBERT Analysis](#SNLI-ALBERT-Analysis)
5. [SNLI XLNET Analysis](#SNLI-XLNET-Analysis)
6. [MNLI Baseline Analysis](#MNLI-Baseline-Analysis)
7. [MNLI BERT Analysis](#MNLI-BERT-Analysis)
8. [MNLI RoBERTa Analysis](#MNLI-RoBERTa-Analysis)
9. [MNLI ALBERT Analysis](#MNLI-ALBERT-Analysis)
10. [MNLI XLNET Analysis](#MNLI-XLNET-Analysis)

In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from IPython.display import display, HTML, Markdown 
from lr.analysis.util import get_ts_from_results_folder 
from lr.stats.h_testing import get_paired_t_statistic
from matplotlib.ticker import FuncFormatter
from glob import glob
from lr.text_processing.util import pre_process_nli_df, get_corpus, get_vocab_count
from lr.training.util import get_ternary_label, filter_df_by_label
from collections import Counter

### Functions

In [2]:
def count_mismatch(paths, size):
    counter = {i:0 for i in range(size)}
    for path in paths:
        df = pd.read_csv(path)
        mismatch_ids  = list(df[df.A != df.B].index)
        for i in mismatch_ids:
            counter[i]+=1
    counter = Counter(counter)
    
    return counter

def show_qualitative_analysis(data,
                              model,
                              test_data,
                              transormed_test_data,
                              path_syn_noun,
                              header,
                              top_n,
                              number_test,
                              top_words):
    
    display(Markdown(header))
    display(Markdown("\n"))
    size = test_data.shape[0]

    paths = []


    for i in range(1, number_test+1):
        paths += glob("raw_results/{}/{}/syn_p_h/batch{}/*.csv".format(data,model,i))


    ids_mismatch = count_mismatch(paths=paths, size=size)
    ids = [i[0] for i in ids_mismatch.most_common(top_n)]

    display(Markdown("#### Top {} Most Frequent Mismatches".format(top_n)))
    display(Markdown("\n"))
    for i in ids:
        original = test_data.loc[[i],:]
        original.index = ["original"]
        transformed = transormed_test_data.loc[[i],:]
        transformed.index = ["transformed"]
        df = pd.concat([original, transformed])    
        display(HTML(df.to_html()))

    display(Markdown("#### Label Distribution"))
    display(Markdown("\n"))
    label_distr = ((test_data.loc[ids].label.value_counts()/top_n).apply(lambda x : "{:.1%}".format(x))).to_frame()
    display(HTML(label_distr.to_html()))

    display(Markdown("\n"))
    display(Markdown("#### Top {} Most Frequent Translations".format(top_n)))
    display(Markdown("\n"))
    syn_noun = pd.read_csv(path_syn_noun)
    syn_noun_map = {k:v for k,v in zip(syn_noun.key, syn_noun.value)}
    t_count = get_vocab_count(test_data.loc[ids,:])
    noums = []

    for i in t_count:
        if i in syn_noun_map:
            noums.append((i, syn_noun_map[i], t_count[i]))
    noums = pd.DataFrame(noums, columns=["original", "transformed", "count"])
    noums = noums.sort_values("count", ascending=False).reset_index(drop=True).head(top_words)
    display(HTML(noums.to_html()))

## SNLI 

In [3]:
path_snli = "data/snli/test.csv" 
path_snli_t = "data/snli/test_p_h_syn_noun.csv"
path_syn_noun_snli = "data/snli/syn_noun.csv"

test_snli = pd.read_csv(path_snli)
test_snli_t = pd.read_csv(path_snli_t)


test_snli = filter_df_by_label(test_snli.dropna()).reset_index(drop=True)
test_snli_t = filter_df_by_label(test_snli_t.dropna()).reset_index(drop=True)

pre_process_nli_df(test_snli)
pre_process_nli_df(test_snli_t)

In [4]:
show_qualitative_analysis(data="snli",
                          model="xgb",
                          test_data=test_snli,
                          transormed_test_data=test_snli_t,
                          path_syn_noun=path_syn_noun_snli,
                          header="# SNLI-Baseline-Analysis",
                          top_n=10,
                          number_test=5,
                          top_words=10)

# SNLI-Baseline-Analysis

#### Top 10 Most Frequent Mismatches

,premise,hypothesis,label
original,a football layer wearing a red shirt,a football player in a black jersey,contradiction
transformed,a football game bed wearing a red shirt,a football game participant in a black jersey,contradiction


,premise,hypothesis,label
original,a crowd watching a skateboarder,a cow is eating grass,contradiction
transformed,a crowd watching a skateboarder,a moocow is eating grass,contradiction


,premise,hypothesis,label
original,a woman wearing purple and carrying a bag is walking out of a door,a woman was indoors before walking outside,entailment
transformed,a adult female wearing purple and carrying a bag is walking out of a door,a adult female was inside before walking outside,entailment


...

,premise,hypothesis,label
original,a man in a tank top fixing himself a hotdog,two girls were ther,contradiction
transformed,a adult male in a army tank top fixing himself a hot dog,two girls were ther,contradiction


#### Label Distribution

,label
contradiction,50.0%
neutral,30.0%
entailment,20.0%


#### Top 10 Most Frequent Translations

,original,transformed,count
0,man,adult male,3
1,football,football game,2
2,indoors,inside,2
3,runner,smuggler,2
4,ballerina,danseuse,2
5,ride,drive,2
6,woman,adult female,2
7,scooter,sea scooter,1
8,tank,army tank,1
9,sandy,flaxen,1


In [5]:
show_qualitative_analysis(data="snli",
                          model="bert_base",
                          test_data=test_snli,
                          transormed_test_data=test_snli_t,
                          path_syn_noun=path_syn_noun_snli,
                          header="# SNLI-BERT-Analysis",
                          top_n=10,
                          number_test=5,
                          top_words=10)

# SNLI-BERT-Analysis

#### Top 10 Most Frequent Mismatches

,premise,hypothesis,label
original,four women are taking a walk down an icy road,the women are walking on the ice,entailment
transformed,four women are taking a walk down an wintry route,the women are walking on the water ice,entailment


,premise,hypothesis,label
original,a man holding some sort of vegetables while his wife in a red sweater stares to the side and his daughter in a red sweater looks at the camera,a man holds a vegetable,entailment
transformed,a adult male holding some kind of vegetables while his married woman in a red jumper stares to the side and his girl in a red jumper looks at the photographic camera,a adult male holds a veggie,entailment


,premise,hypothesis,label
original,a homeless man carries a sign that says hungry,the man is living in a mansion,contradiction
transformed,a homeless person adult male carries a mark that says hungry,the adult male is living in a sign,contradiction


...

,premise,hypothesis,label
original,the wind propels a sailing ship on a group of cruisers,the ship nearly hits a cruiser,neutral
transformed,the air current propels a sailing ship on a group of cruisers,the ship nearly hits a police car,neutral


#### Label Distribution

,label
entailment,60.0%
neutral,20.0%
contradiction,20.0%


#### Top 10 Most Frequent Translations

,original,transformed,count
0,man,adult male,8
1,market,marketplace,2
2,sweater,jumper,2
3,camera,photographic camera,2
4,treat,dainty,2
5,farmer,granger,2
6,icy,wintry,1
7,wind,air current,1
8,underground,resistance,1
9,cowboy,puncher,1


In [6]:
show_qualitative_analysis(data="snli",
                          model="roberta_base",
                          test_data=test_snli,
                          transormed_test_data=test_snli_t,
                          path_syn_noun=path_syn_noun_snli,
                          header="# SNLI-RoBERTa-Analysis",
                          top_n=10,
                          number_test=5,
                          top_words=10)

# SNLI-RoBERTa-Analysis

#### Top 10 Most Frequent Mismatches

,premise,hypothesis,label
original,the woman in the black shirt hands papers to the children surrounding her,the woman is giving homework to the children,neutral
transformed,the adult female in the black shirt hands document to the child surrounding her,the adult female is giving preparation to the child,neutral


,premise,hypothesis,label
original,boys with their backs against an incoming wave,a group of people play in the ocean,neutral
transformed,boys with their backs against an incoming moving ridge,a group of people play in the ocean,neutral


,premise,hypothesis,label
original,a group of people look at fresh lettuce in at the local street market,people viewing produce at a market,entailment
transformed,a group of people look at fresh bread in at the local street marketplace,people viewing garden truck at a marketplace,entailment


...

,premise,hypothesis,label
original,xylophonists play in a dark setting,someone who plays a xylophone,entailment
transformed,xylophonists play in a dark setting,someone who plays a marimba,entailment


#### Label Distribution

,label
entailment,50.0%
neutral,40.0%
contradiction,10.0%


#### Top 10 Most Frequent Translations

,original,transformed,count
0,man,adult male,5
1,woman,adult female,3
2,children,child,2
3,market,marketplace,2
4,produce,garden truck,2
5,wheelbarrow,barrow,2
6,court,judicature,1
7,person,individual,1
8,individual,person,1
9,bridal,espousal,1


In [7]:
show_qualitative_analysis(data="snli",
                          model="albert_base",
                          test_data=test_snli,
                          transormed_test_data=test_snli_t,
                          path_syn_noun=path_syn_noun_snli,
                          header="# SNLI-ALBERT-Analysis",
                          top_n=10,
                          number_test=5,
                          top_words=10)

# SNLI-ALBERT-Analysis

#### Top 10 Most Frequent Mismatches

,premise,hypothesis,label
original,two girls kissing a man with a black shirt and brown hair on the cheeks,two girls kiss,contradiction
transformed,two girls kissing a adult male with a black shirt and brown hair on the cheeks,two girls buss,contradiction


,premise,hypothesis,label
original,a young boy in green playing pingpong with a older man in blue in a gymnasium,they are playing ping pong,entailment
transformed,a young male child in green playing pingpong with a older adult male in blue in a gymnasium,they are playing ping niff,entailment


,premise,hypothesis,label
original,three musicians are playing in a band,the band consists of a trio of musicians,entailment
transformed,three player are playing in a band,the band lie of a leash of player,entailment


...

,premise,hypothesis,label
original,a nighttime street scene of a restaurant,the restaurant is open at night,entailment
transformed,a night street scene of a eatery,the eatery is clear at dark,entailment


#### Label Distribution

,label
entailment,80.0%
contradiction,20.0%


#### Top 10 Most Frequent Translations

,original,transformed,count
0,man,adult male,8
1,woman,adult female,4
2,kiss,buss,3
3,buy,steal,2
4,open,clear,2
5,musicians,player,2
6,restaurant,eatery,2
7,baby,infant,2
8,market,marketplace,2
9,nighttime,night,1


In [8]:
show_qualitative_analysis(data="snli",
                          model="xlnet_base",
                          test_data=test_snli,
                          transormed_test_data=test_snli_t,
                          path_syn_noun=path_syn_noun_snli,
                          header="# SNLI-XLNET-Analysis",
                          top_n=10,
                          number_test=5,
                          top_words=10)

# SNLI-XLNET-Analysis

#### Top 10 Most Frequent Mismatches

,premise,hypothesis,label
original,a boy in a red hooded top is smiling whilst looking away from his reflection,a boy in a red hood is happy,entailment
transformed,a male child in a red hooded top is smiling whilst looking away from his contemplation,a male child in a red punk is happy,entailment


,premise,hypothesis,label
original,four women are taking a walk down an icy road,the women are walking on the ice,entailment
transformed,four women are taking a walk down an wintry route,the women are walking on the water ice,entailment


,premise,hypothesis,label
original,a group of people look at fresh lettuce in at the local street market,people viewing produce at a market,entailment
transformed,a group of people look at fresh bread in at the local street marketplace,people viewing garden truck at a marketplace,entailment


...

,premise,hypothesis,label
original,a couple about to kiss somewhere,a couple are kissing on new years eve as the ball drops,neutral
transformed,a duo about to buss somewhere,a duo are kissing on new years eve as the ball drops,neutral


#### Label Distribution

,label
entailment,80.0%
neutral,20.0%


#### Top 10 Most Frequent Translations

,original,transformed,count
0,man,adult male,6
1,market,marketplace,2
2,couple,duo,2
3,woman,adult female,2
4,golf,golf game,2
5,boy,male child,2
6,hood,punk,2
7,lettuce,bread,1
8,ice,water ice,1
9,chat,confab,1


## MNLI 

In [9]:
path_mnli = "data/mnli/test.csv" 
path_mnli_t = "data/mnli/test_p_h_syn_noun.csv"
path_syn_noun_mnli = "data/mnli/syn_noun.csv"

test_mnli = pd.read_csv(path_mnli)
test_mnli_t = pd.read_csv(path_mnli_t)


test_mnli = filter_df_by_label(test_mnli.dropna()).reset_index(drop=True)
test_mnli_t = filter_df_by_label(test_mnli_t.dropna()).reset_index(drop=True)

pre_process_nli_df(test_mnli)
pre_process_nli_df(test_mnli_t)

In [10]:
show_qualitative_analysis(data="mnli",
                          model="xgb",
                          test_data=test_mnli,
                          transormed_test_data=test_mnli_t,
                          path_syn_noun=path_syn_noun_mnli,
                          header="# MNLI-Baseline-Analysis",
                          top_n=10,
                          number_test=5,
                          top_words=10)

# MNLI-Baseline-Analysis

#### Top 10 Most Frequent Mismatches

,premise,hypothesis,label
original,indeed 58 percent of columbiahcas beds lie empty compared with 35 percent of nonprofit beds,58 of columbiahcas beds are empty said the report,entailment
transformed,indeed 58 percentage of columbiahcas beds prevarication empty compared with 35 percentage of notforprofit beds,58 of columbiahcas beds are empty state the study,entailment


,premise,hypothesis,label
original,indeed 58 percent of columbiahcas beds lie empty compared with 35 percent of nonprofit beds,58 of columbiahcas beds are full,contradiction
transformed,indeed 58 percentage of columbiahcas beds prevarication empty compared with 35 percentage of notforprofit beds,58 of columbiahcas beds are full,contradiction


,premise,hypothesis,label
original,indeed 58 percent of columbiahcas beds lie empty compared with 35 percent of nonprofit beds,58 of columbiahcas beds are empty,entailment
transformed,indeed 58 percentage of columbiahcas beds prevarication empty compared with 35 percentage of notforprofit beds,58 of columbiahcas beds are empty,entailment


...

,premise,hypothesis,label
original,in their description of isoglosses they,they omit any description of isoglosses,contradiction
transformed,in their verbal description of isoglosses they,they omit any verbal description of isoglosses,contradiction


#### Label Distribution

,label
entailment,50.0%
contradiction,40.0%
neutral,10.0%


#### Top 10 Most Frequent Translations

,original,transformed,count
0,percent,percentage,6
1,nonprofit,not-for-profit,4
2,territory,district,4
3,standard,measure,3
4,days,years,3
5,lie,prevarication,3
6,program,plan,2
7,gum,chewing gum,2
8,death,decease,2
9,before,earlier,2


In [11]:
show_qualitative_analysis(data="mnli",
                          model="bert_base",
                          test_data=test_mnli,
                          transormed_test_data=test_mnli_t,
                          path_syn_noun=path_syn_noun_mnli,
                          header="# MNLI-BERT-Analysis",
                          top_n=10,
                          number_test=5,
                          top_words=10)

# MNLI-BERT-Analysis

#### Top 10 Most Frequent Mismatches

,premise,hypothesis,label
original,auditors are strongly encouraged to comply with the guidance provided by gagas,gagas offers guidance to auditors for compliance purposes,entailment
transformed,auditors are strongly encouraged to follow with the direction provided by gagas,gagas offers direction to auditors for conformity design,entailment


,premise,hypothesis,label
original,in manual systems attestations verifications and approvals are usually shown by a signature or initial of an individual on a hard copy document,signatures in manual systems usually show approval,entailment
transformed,in manual systems attestations check and approvals are usually shown by a signature or initial of an person on a difficult transcript papers,signatures in manual systems usually show blessing,entailment


,premise,hypothesis,label
original,its come back cried julius excitedly,they thought it was gone forever,neutral
transformed,its come back cried julius excitedly,they idea it was travel forever,neutral


...

,premise,hypothesis,label
original,a diskette is also called a floppy the reason for the name is not immediately apparent nor why the item is called a diskette for that matter but all becomes clear,diskettes may also be referred to as floppies,entailment
transformed,a floppy is too called a diskette the ground for the name is not immediately apparent nor why the point is called a floppy for that thing but all go clear,diskettes may too be referred to as diskette,entailment


#### Label Distribution

,label
entailment,80.0%
neutral,20.0%


#### Top 10 Most Frequent Translations

,original,transformed,count
0,market,marketplace,3
1,gone,travel,3
2,mother,female parent,2
3,minutes,transactions,2
4,share,part,2
5,manufacturers,maker,2
6,entertainment,amusement,2
7,percent,percentage,2
8,operations,trading operations,2
9,diskette,floppy,2


In [12]:
show_qualitative_analysis(data="mnli",
                          model="roberta_base",
                          test_data=test_mnli,
                          transormed_test_data=test_mnli_t,
                          path_syn_noun=path_syn_noun_mnli,
                          header="# MNLI-RoBERTa-Analysis",
                          top_n=10,
                          number_test=5,
                          top_words=10)

# MNLI-RoBERTa-Analysis

#### Top 10 Most Frequent Mismatches

,premise,hypothesis,label
original,well so okay you need to get married and have kids and then when theyre big enough you can have them go do the yard and you can do what you want to do,the reason to have children is so that they can do yardwork,entailment
transformed,well so okey you demand to get married and have child and then when theyre large sufficiency you can have them go do the pace and you can do what you want to do,the ground to have child is so that they can do yardwork,entailment


,premise,hypothesis,label
original,in the vaults of the bank,in the bank vault,entailment
transformed,in the vaults of the bank,in the bank burial vault,entailment


,premise,hypothesis,label
original,ssa is also seeking statutory authority for additional tools to recover current overpayments,ssa wants the authority to recover overpayments,entailment
transformed,ssa is too seeking statutory authority for additional tools to find electric current overpayments,ssa deprivation the authority to find overpayments,entailment


...

,premise,hypothesis,label
original,adrins third lesson,adrin had three lessons,neutral
transformed,adrins onethird lesson,adrin had three lessons,neutral


#### Label Distribution

,label
entailment,80.0%
neutral,10.0%
contradiction,10.0%


#### Top 10 Most Frequent Translations

,original,transformed,count
0,also,too,2
1,agencies,office,2
2,need,demand,2
3,essential,requirement,2
4,quote,quotation mark,2
5,recover,find,2
6,expertise,expertness,2
7,circumstances,lot,2
8,said,state,1
9,bit,spot,1


In [13]:
show_qualitative_analysis(data="mnli",
                          model="albert_base",
                          test_data=test_mnli,
                          transormed_test_data=test_mnli_t,
                          path_syn_noun=path_syn_noun_mnli,
                          header="# MNLI-ALBERT-Analysis",
                          top_n=10,
                          number_test=5,
                          top_words=10)

# MNLI-ALBERT-Analysis

#### Top 10 Most Frequent Mismatches

,premise,hypothesis,label
original,piccadilly tube station,piccadilly subway station,entailment
transformed,piccadilly tube station,piccadilly tube station,entailment


,premise,hypothesis,label
original,ssa is also seeking statutory authority for additional tools to recover current overpayments,ssa wants the authority to recover overpayments,entailment
transformed,ssa is too seeking statutory authority for additional tools to find electric current overpayments,ssa deprivation the authority to find overpayments,entailment


,premise,hypothesis,label
original,let me emphasize by further extending in the quote above,the quote above will show the emphasis,entailment
transformed,let me stress by further extending in the quotation mark above,the quotation mark above will show the accent,entailment


...

,premise,hypothesis,label
original,proposition 3 the assembly roomthe traditional focus of attention for industry competitivenesscan provide competitive benefits only if other more fundamental changes in manufacturing practice have been introduced,changing the manufacturing practice allows the industry to focus on competitiveness,entailment
transformed,proposition 3 the assembly roomthe traditional direction of attention for industry competitivenesscan supply competitive benefits only if other more fundamental changes in manufacturing pattern have be introduced,changing the manufacturing pattern allows the industry to direction on fight,entailment


#### Label Distribution

,label
entailment,80.0%
neutral,10.0%
contradiction,10.0%


#### Top 10 Most Frequent Translations

,original,transformed,count
0,feminist,libber,3
1,dictionary,lexicon,2
2,recover,find,2
3,practice,pattern,2
4,quote,quotation mark,2
5,focus,direction,2
6,grand,yard,2
7,palace,castle,2
8,also,too,2
9,guidance,direction,2


In [14]:
show_qualitative_analysis(data="mnli",
                          model="xlnet_base",
                          test_data=test_mnli,
                          transormed_test_data=test_mnli_t,
                          path_syn_noun=path_syn_noun_mnli,
                          header="# MNLI-XLNET-Analysis",
                          top_n=10,
                          number_test=5,
                          top_words=10)

# MNLI-XLNET-Analysis

#### Top 10 Most Frequent Mismatches

,premise,hypothesis,label
original,auditors are strongly encouraged to comply with the guidance provided by gagas,gagas offers guidance to auditors for compliance purposes,entailment
transformed,auditors are strongly encouraged to follow with the direction provided by gagas,gagas offers direction to auditors for conformity design,entailment


,premise,hypothesis,label
original,perhaps tax reform doesnt appeal to the new spiritualized side of bradley,bradley is spiritual now,entailment
transformed,perhaps taxation reform doesnt prayer to the new spiritualized side of bradley,bradley is negro spiritual now,entailment


,premise,hypothesis,label
original,an arresting one occurs in a passage to india chapter xxi in which forster describes a small building as a flimsy and frivolous erection while in the mayor of casterbridge chapter xvi the mayor himself beheld the unattractive exterior of farfraes erection,in a passage to india buildings are described as erections,entailment
transformed,an arresting one pass in a transition to india chapter xxi in which forster draw a small building as a onionskin and frivolous hardon while in the city manager of casterbridge chapter xvi the city manager himself beheld the unattractive outside of farfraes hardon,in a transition to india buildings are described as erections,entailment


...

,premise,hypothesis,label
original,working groups were established to coordinate training statewide to focus on the establishment of a statewide website and to continue coordination and sharing in technology matters,groups were formed to coordinate training around the state,entailment
transformed,working groups were established to coordinate training statewide to direction on the organization of a statewide site and to continue coordination and sharing in engineering thing,groups were formed to coordinate training around the province,entailment


#### Label Distribution

,label
entailment,90.0%
contradiction,10.0%


#### Top 10 Most Frequent Translations

,original,transformed,count
0,technology,engineering,3
1,mayor,city manager,2
2,grand,yard,2
3,settlement,colony,2
4,also,too,2
5,diskette,floppy,2
6,guidance,direction,2
7,erection,hard-on,2
8,focus,direction,2
9,passage,transition,2
